# Using IMF Currency Data

The [International Monetary Fund (IMF)](https://www.imf.org) publishes a range of time series data on IMF lending, exchange rates and other economic and financial indicators.

In this template we show how to to use currency data for developing a trading algorithm.

**Need help?** Check the [**Documentation**](https://quantiacs.com/documentation/en/) and find solutions/report problems in the [**Forum**](https://quantiacs.com/community/categories) section.

**More help with Jupyter?** Check the official [**Jupyter**](https://jupyter.org/) page.

**Documentation on the IMF data** can be found [**here**](https://github.com/quantiacs/documentation/blob/master/en/source/user_guide/data.md).

Once you are done, click on **Submit to the contest** and take part to our competitions.

API reference:

* **data**: check how to work with [data](https://quantiacs.com/documentation/en/reference/data_load_functions.html);

* **backtesting**: read how to run the [simulation](https://quantiacs.com/documentation/en/reference/evaluation.html) and check the results.

In this template we use the optimizer function described in:

* **optimization**: read more on our [article](https://quantiacs.com/community/topic/29/optimizing-and-monitoring-a-trading-system-with-quantiacs).

In [ ]:
%%javascript
window.IPython && (IPython.OutputArea.prototype._should_scroll = function(lines) { return false; })
// disable widget scrolling

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata

In [ ]:
# currencies listing
currency_list = qndata.imf_load_currency_list()
pd.DataFrame(currency_list)

In [ ]:
def load_data(period):
    # load the AEX Index data and the spot EUR rate:
    futures  = qndata.futures_load_data(assets=['F_AE'], tail=period, dims=('time','field','asset'))
    currency = qndata.imf_load_currency_data(assets=['EUR'], tail=period).isel(asset=0)
    return dict(currency=currency, futures=futures), futures.time.values


def window(data, max_date: np.datetime64, lookback_period: int):
    # build sliding window for rolling evaluation:
    min_date = max_date - np.timedelta64(lookback_period, 'D')
    return dict(
        futures  = data['futures'].sel(time=slice(min_date, max_date)),
        currency = data['currency'].sel(time=slice(min_date, max_date))
    )


def strategy(data):
    # this strategy uses the currency data as predictors for the Futures contract:   
    close = data['futures'].sel(field='close')
    currency = data['currency']
    
    ma1 = qnta.lwma(currency,10)
    ma2 = qnta.lwma(currency,50)
    ma3 = qnta.lwma(currency,250)
    
    if ma1.isel(time=-1) > ma2.isel(time=-1) and ma2.isel(time=-1) < ma3.isel(time=-1):
        return xr.ones_like(close.isel(time=-1))
    else:
        return xr.zeros_like(close.isel(time=-1))


weights = qnbt.backtest(
    competition_type='futures',
    load_data=load_data,
    window=window,
    lookback_period=365,
    start_date='2006-01-01',
    strategy=strategy,
    analyze=True,
    build_plots=True
)